In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install sentence_transformers
!pip install transformers
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 88.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.3 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=994c148600f60478c8cf2a4224e4ff1ae24d19a98332de51cd8139da3720e1ff
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.9 MB/

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
class CustomDataset(Dataset):
    def __init__(self, additional_data):
        self.additional_data = additional_data
        self.tokenizer =  AutoTokenizer.from_pretrained("vinai/phobert-large")


    def __len__(self):
        return len(self.additional_data)

    def __getitem__(self, idx):
        sentence_pair = self.additional_data[idx]
        claim, context, evidence = sentence_pair['claim'], sentence_pair['context'], sentence_pair['evidence']

        #mapped_label = self.label_map[label]

        tokenized_inputs = self.tokenizer.batch_encode_plus(
            (claim + " " + context, evidence) ,
            padding="longest",
            truncation=True,
            return_tensors="pt",
          )

        return {

            "input_ids": tokenized_inputs["input_ids"],
            "attention_mask":tokenized_inputs["attention_mask"],
            "target_ids":  tokenized_inputs["input_ids"]
        }

In [ ]:
#load data
import pickle
train_dataloader_path = "/content/drive/MyDrive/DSC/DSC/notebook/TrainSeq2Seq.pkl"  # Specify the path to save the train DataLoader
test_dataloader_path = "/content/drive/MyDrive/DSC/DSC/notebook/testSeq2Seq.pkl"


with open(train_dataloader_path, "rb") as f_train:
    train_data_loader = pickle.load(f_train)

with open(test_dataloader_path, "rb") as f_test:
    test_data_loader = pickle.load(f_test)

In [ ]:
train_data = CustomDataset(train_data_loader)
test_data = CustomDataset(test_data_loader)

In [ ]:
train_data[4]

{'input_ids': tensor([[    0,    79,  5019, 20792,   350, 10029,     4,   185,  1038,    84,
             67,  3959,    15,   626,  5924,  2199,  5214,   230,  2767,   380,
            136,     8,  2555,   113, 17806,  1701,  7269,    78,    35,  1696,
           7657,     6,  6342,   113, 17806,  1701,  7269,    78,    35,   253,
            383, 33784,    15,     9,    69,  2054, 26427,  1395,     4,  5924,
          26286,  1395,     4,   184,  2379,  2209,  2153,  1889, 53781,  1187,
              4,   292, 32000, 12151,     8,  1783,  3139,  2665,  7594,  4436,
           2608,    61,   302,   675,     7,   675,  1893, 25925,  1098,     6,
             61,  1073, 12155,  1560, 18232,  1301,     4,  6169,  7311,  3959,
             15,     9,  7362,  5191,  1652,   447,   675,  5116, 34622,  2586,
              3, 30939,   218,    25,    44,    93,    90,    61, 14341,  3984,
           2054, 26427,  1395, 30939,     4, 10969,  5214,  6713,  2276, 40780,
           1701,     4,   5

In [ ]:
from transformers import  AutoModelForSeq2SeqLM, AutoConfig
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer
from transformers import get_scheduler, get_cosine_schedule_with_warmup
from torch.optim import AdamW

# Load pre-trained XLM-R model
model_name = "vinai/bartpho-syllable"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer =  AutoTokenizer.from_pretrained(model_name)





model.config.decoder_start_token_id = tokenizer.pad_token_id
model.config.pad_token_id = tokenizer.pad_token_id

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="content/seq2seqModel",
    num_train_epochs=15,
    learning_rate= 2e-6,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=1000,
    logging_dir="./logs",
    logging_steps=100,
    save_steps=500,
    evaluation_strategy="steps",
    eval_steps=500,
)


optimizer = AdamW(model.parameters(), lr=training_args.learning_rate)
num_training_steps = len(train_data) // training_args.per_device_train_batch_size * training_args.num_train_epochs
scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=1000,
    num_training_steps=num_training_steps

)

# Create the Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer,
    optimizers= (optimizer,scheduler) ,
)




In [ ]:
trainer.train()
trainer.save_model("content/seq2seqModel")

In [ ]:
!unzip /content/drive/MyDrive/DSC/DSC/Seq2SeqModel/seq2seqModel.zip -d seq2seq

Archive:  /content/drive/MyDrive/DSC/DSC/Seq2SeqModel/seq2seqModel.zip
 extracting: seq2seq/config.json     
 extracting: seq2seq/generation_config.json  
 extracting: seq2seq/pytorch_model.bin  
 extracting: seq2seq/tokenizer_config.json  
 extracting: seq2seq/special_tokens_map.json  
 extracting: seq2seq/spiece.model    
 extracting: seq2seq/tokenizer.json  
 extracting: seq2seq/training_args.bin  


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
from transformers import AutoTokenizer

model_checkpoint = "/content/seq2seq"
#model_checkpoint = "nguyenvulebinh/vi-mrc-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:



context ="TS Trần Mạnh Hà, Trưởng phòng Đào tạo, hôm 28/3 cho biết bốn chương trình đào tạo mới gồm: Ngân hàng số, Công nghệ tài chính (Fintech), Logistics và Quản lý chuỗi cung ứng, Quản trị du lịch. Việc này, theo ông Hà nhằm đáp ứng nhu cầu nhân lực cho nền kinh tế trong giai đoạn tới.\n\nTổng chỉ tiêu tuyển sinh của Học viện Ngân hàng là 3.300, tăng khoảng 100 so với năm 2022. Chỉ tiêu, tổ hợp xét tuyển từng ngành như sau:\n\nTheo ông Hà, phương thức xét tuyển của Học viện Ngân hàng năm 2023 tương tự như năm 2022, gồm: xét học bạ THPT (dự kiến 25% tổng chỉ tiêu), chứng chỉ quốc tế (15%), kết quả kỳ thi đánh giá năng lực (10%), kết quả thi tốt nghiệp THPT (50%) và xét tuyển thẳng.\n\nĐiểm xét tuyển là tổng điểm trung bình cộng các năm của ba môn trong tổ hợp cộng điểm ưu tiên, khuyến khích. Riêng các chương trình chất lượng cao, điểm được tính trên thang 40 với môn Toán nhân hệ số hai. Đây là điểm mới so với năm ngoái.\n\nThí sinh được thêm 1,5-3 điểm khuyến khích nếu có giải trong các cuộc thi học sinh giỏi cấp tỉnh, quốc gia hoặc là học sinh hệ chuyên của trường THPT chuyên.\n\nĐiểm xét tuyển = (Điểm kết quả đánh giá năng lực : 150) x 30 + Điểm ưu tiên.\n\nHọc viện Ngân hàng dự kiến thu hồ sơ với các phương thức trên và xét tuyển thẳng trong hai tuần đầu tháng 6. Danh sách thí sinh đủ điều kiện trúng tuyển sẽ được thông báo vào cuối tháng 6.\n\nNăm ngoái, điểm chuẩn Học viện Ngân hàng theo phương thức xét kết quả thi tốt nghiệp THPT là từ 24 đến 28,05, trong đó ngành Luật kinh tế có đầu vào cao nhất."
question=  "Việc có thêm chương trình đào tạo mới, theo ông Hà nhằm đáp ứng nhu cầu nhân lực cho nền kinh tế trong giai đoạn tới."
ground_truth ="TS Trần Mạnh Hà, Trưởng phòng Đào tạo, hôm 28/3 cho biết bốn chương trình đào tạo mới gồm: Ngân hàng số, Công nghệ tài chính (Fintech), Logistics và Quản lý chuỗi cung ứng, Quản trị du lịch",



input_ids = tokenizer.encode(question+ " "+ context , return_tensors='pt')
output = model.generate(input_ids, max_length=128)
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded_output)

Việc này, theo ông Hà nhằm đáp ứng nhu cầu nhân lực cho nền kinh tế trong giai đoạn tới.


In [ ]:

submit_data="/content/drive/MyDrive/DSC/DSC/DSC/Data/ise-dsc01-public-test-offcial (1).json"
with open(submit_data , encoding='utf-8') as fh:
    data = json.load(fh)

for key in tqdm(data.keys()):
  claim = data[key]["claim"]
  context = data[key]["context"]

  QA_input = {
  'question': claim,
  'context': context
  }



  res = nlp(QA_input)
  print('pipeline: {}'.format(res))



  0%|          | 1/4794 [00:09<12:56:11,  9.72s/it]

pipeline: {'score': 0.4511053264141083, 'start': 683, 'end': 692, 'answer': 'là nguyên'}


  0%|          | 2/4794 [00:14<9:14:00,  6.94s/it] 

pipeline: {'score': 0.14717312157154083, 'start': 1209, 'end': 1239, 'answer': 'sự nhạy bén trong phân tích dữ'}


  0%|          | 2/4794 [00:34<22:40:38, 17.04s/it]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-5e7b4f25ab97>", line 16, in <cell line: 5>
    res = nlp(QA_input)
  File "/usr/local/lib/python3.10/dist-packages/transformers/pipelines/question_answering.py", line 393, in __call__
    return super().__call__(examples[0], **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py", line 1132, in __call__
    return next(
  File "/usr/local/lib/python3.10/dist-packages/transformers/pipelines/pt_utils.py", line 124, in __next__
    item = next(self.iterator)
  File "/usr/local/lib/python3.10/dist-packages/transformers/pipelines/pt_utils.py", line 266, in __next__
    processed = self.infer(next(self.iterator), **self.params)
  File "/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py", line 1046, in forward
    model_out

TypeError: ignored

In [ ]:
from transformers import XLMRobertaTokenizer

tokenizer = XLMRobertaTokenizer.from_pretrained("content/seq2seqModel")

# Tokenize source (claims + context) and target (evidence) sequences
tokenized_inputs = tokenizer.batch_encode_plus(
    [(claim + " " + context, evidence) for claim, context, evidence in dataset],
    padding="longest",
    truncation=True,
    return_tensors="pt",
)

input_ids = tokenized_inputs["input_ids"]
attention_mask = tokenized_inputs["attention_mask"]
target_ids = tokenized_inputs["input_ids"]

In [ ]:
model.eval()
generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=50)
generated_evidence = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

In [ ]:
import json
submit_data="/content/drive/MyDrive/DSC/results/public_result_Khang.json"
with open(submit_data , encoding='utf-8') as fh:
    first_data = json.load(fh)
count ={"SUPPORTED": 0,
        "NEI": 0 ,
        "REFUTED": 0}

for key in first_data.keys():
  value = first_data[key]
  count[value['verdict']] +=1
count

{'SUPPORTED': 1430, 'NEI': 1618, 'REFUTED': 1746}

In [ ]:
import json
from tqdm import tqdm
submit_data="/content/drive/MyDrive/DSC/results/public_result_dat.json"
with open(submit_data , encoding='utf-8') as fh:
    best_data = json.load(fh)
count ={"SUPPORTED": 0,
        "NEI": 0 ,
        "REFUTED": 0}

for key in  best_data.keys():
  value =  best_data[key]
  count[value['verdict']] +=1
count

{'SUPPORTED': 1522, 'NEI': 1516, 'REFUTED': 1756}

In [ ]:
#mix two data
result= {}

for key in tqdm(best_data.keys()):
  result[key]= {}
  label = best_data[key]["verdict"]
  evidence = first_data[key]["evidence"]



  if label =="NEI":
    result[key]["verdict"]= label
    result[key]["evidence"]= ""
  else:
    result[key]["verdict"]= label
    result[key]["evidence"]= evidence




100%|██████████| 4794/4794 [00:00<00:00, 576922.89it/s]


In [ ]:
import json

count ={"SUPPORTED": 0,
        "NEI": 0 ,
        "REFUTED": 0}

for key in result.keys():
  value =result[key]
  count[value['verdict']] +=1
count

{'SUPPORTED': 1522, 'NEI': 1516, 'REFUTED': 1756}

In [ ]:
with open("public_result.json", "w") as outfile:
    json.dump(result, outfile)